In [173]:
# Import packages
import pandas as pd
import numpy as np
import os
import pyspark
from pathlib import Path
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, unix_timestamp, when

# Make pandas dataframes prettier
from IPython.display import display, HTML

In [2]:
# Initialise PySpark context and session
sc = pyspark.SparkContext(appName="appName")
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("appName").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/08 14:20:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Loading and exploring data

In [207]:
# Helper code to unzip DIAGNOSES_ICD.csv.gz

# import gzip
# import shutil
# with gzip.open(mimic_dir + 'DIAGNOSES_ICD.csv.gz', 'rb') as f_in:
#    with open(mimic_dir + 'DIAGNOSES_ICD.csv', 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)

In [3]:
# Load dataset using PySpark
mimic_dir = "../../data/mimic-iii/"

adm_df = spark.read.option("header", True).option("inferSchema", True).csv(
    mimic_dir + "ADMISSIONS.csv")
chart_df = spark.read.option("header", True).option("inferSchema", True).csv(
    mimic_dir + "CHARTEVENTS.csv")
ditems_df = spark.read.option("header", True).option("inferSchema", True).csv(
    mimic_dir + "D_ITEMS.csv")
icu_df = spark.read.option("header", True).option("inferSchema", True).csv(
    mimic_dir + "ICUSTAYS.csv")
pat_df = spark.read.option("header", True).option("inferSchema", True).csv(
    mimic_dir + "PATIENTS.csv")
diag_df = spark.read.option("header", True).option("inferSchema", True).csv(
    mimic_dir + "DIAGNOSES_ICD.csv")

23/07/08 14:20:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
# Check schema for admission table
adm_df.printSchema()

root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ADMITTIME: timestamp (nullable = true)
 |-- DISCHTIME: timestamp (nullable = true)
 |-- DEATHTIME: timestamp (nullable = true)
 |-- ADMISSION_TYPE: string (nullable = true)
 |-- ADMISSION_LOCATION: string (nullable = true)
 |-- DISCHARGE_LOCATION: string (nullable = true)
 |-- INSURANCE: string (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- RELIGION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- EDREGTIME: timestamp (nullable = true)
 |-- EDOUTTIME: timestamp (nullable = true)
 |-- DIAGNOSIS: string (nullable = true)
 |-- HOSPITAL_EXPIRE_FLAG: integer (nullable = true)
 |-- HAS_CHARTEVENTS_DATA: integer (nullable = true)



In [12]:
# Print first few rows for selected fields
adm_df.select("SUBJECT_ID", "HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME").show(10)

+----------+-------+-------------------+-------------------+-------------------+
|SUBJECT_ID|HADM_ID|          ADMITTIME|          DISCHTIME|          DEATHTIME|
+----------+-------+-------------------+-------------------+-------------------+
|        22| 165315|2196-04-09 12:26:00|2196-04-10 15:54:00|               null|
|        23| 152223|2153-09-03 07:15:00|2153-09-08 19:10:00|               null|
|        23| 124321|2157-10-18 19:34:00|2157-10-25 14:00:00|               null|
|        24| 161859|2139-06-06 16:14:00|2139-06-09 12:48:00|               null|
|        25| 129635|2160-11-02 02:06:00|2160-11-05 14:55:00|               null|
|        26| 197661|2126-05-06 15:16:00|2126-05-13 15:00:00|               null|
|        27| 134931|2191-11-30 22:16:00|2191-12-03 14:45:00|               null|
|        28| 162569|2177-09-01 07:15:00|2177-09-06 16:00:00|               null|
|        30| 104557|2172-10-14 14:17:00|2172-10-19 14:37:00|               null|
|        31| 128652|2108-08-

In [13]:
adm_df.select("SUBJECT_ID", "HADM_ID", "ADMISSION_TYPE", 
              "ADMISSION_LOCATION", "DISCHARGE_LOCATION").show(10)

+----------+-------+--------------+--------------------+--------------------+
|SUBJECT_ID|HADM_ID|ADMISSION_TYPE|  ADMISSION_LOCATION|  DISCHARGE_LOCATION|
+----------+-------+--------------+--------------------+--------------------+
|        22| 165315|     EMERGENCY|EMERGENCY ROOM ADMIT|DISC-TRAN CANCER/...|
|        23| 152223|      ELECTIVE|PHYS REFERRAL/NOR...|    HOME HEALTH CARE|
|        23| 124321|     EMERGENCY|TRANSFER FROM HOS...|    HOME HEALTH CARE|
|        24| 161859|     EMERGENCY|TRANSFER FROM HOS...|                HOME|
|        25| 129635|     EMERGENCY|EMERGENCY ROOM ADMIT|                HOME|
|        26| 197661|     EMERGENCY|TRANSFER FROM HOS...|                HOME|
|        27| 134931|       NEWBORN|PHYS REFERRAL/NOR...|                HOME|
|        28| 162569|      ELECTIVE|PHYS REFERRAL/NOR...|    HOME HEALTH CARE|
|        30| 104557|        URGENT|TRANSFER FROM HOS...|    HOME HEALTH CARE|
|        31| 128652|     EMERGENCY|TRANSFER FROM HOS...|        

In [14]:
adm_df.select("SUBJECT_ID", "HADM_ID", "INSURANCE", "LANGUAGE", 
              "RELIGION", "MARITAL_STATUS", "ETHNICITY").show(10)

+----------+-------+---------+--------+-----------------+--------------+--------------------+
|SUBJECT_ID|HADM_ID|INSURANCE|LANGUAGE|         RELIGION|MARITAL_STATUS|           ETHNICITY|
+----------+-------+---------+--------+-----------------+--------------+--------------------+
|        22| 165315|  Private|    null|     UNOBTAINABLE|       MARRIED|               WHITE|
|        23| 152223| Medicare|    null|         CATHOLIC|       MARRIED|               WHITE|
|        23| 124321| Medicare|    ENGL|         CATHOLIC|       MARRIED|               WHITE|
|        24| 161859|  Private|    null|PROTESTANT QUAKER|        SINGLE|               WHITE|
|        25| 129635|  Private|    null|     UNOBTAINABLE|       MARRIED|               WHITE|
|        26| 197661| Medicare|    null|         CATHOLIC|        SINGLE|UNKNOWN/NOT SPECI...|
|        27| 134931|  Private|    null|         CATHOLIC|          null|               WHITE|
|        28| 162569| Medicare|    null|         CATHOLIC|   

In [15]:
adm_df.select("SUBJECT_ID", "HADM_ID", "EDREGTIME", "EDOUTTIME", "DIAGNOSIS").show(10)

+----------+-------+-------------------+-------------------+--------------------+
|SUBJECT_ID|HADM_ID|          EDREGTIME|          EDOUTTIME|           DIAGNOSIS|
+----------+-------+-------------------+-------------------+--------------------+
|        22| 165315|2196-04-09 10:06:00|2196-04-09 13:24:00|BENZODIAZEPINE OV...|
|        23| 152223|               null|               null|CORONARY ARTERY D...|
|        23| 124321|               null|               null|          BRAIN MASS|
|        24| 161859|               null|               null|INTERIOR MYOCARDI...|
|        25| 129635|2160-11-02 01:01:00|2160-11-02 04:27:00|ACUTE CORONARY SY...|
|        26| 197661|               null|               null|              V-TACH|
|        27| 134931|               null|               null|             NEWBORN|
|        28| 162569|               null|               null|CORONARY ARTERY D...|
|        30| 104557|               null|               null|UNSTABLE ANGINA\CATH|
|        31| 128

In [18]:
adm_df.select("SUBJECT_ID", "HADM_ID", "DEATHTIME", "HOSPITAL_EXPIRE_FLAG", 
              "HAS_CHARTEVENTS_DATA").show(10)

+----------+-------+-------------------+--------------------+--------------------+
|SUBJECT_ID|HADM_ID|          DEATHTIME|HOSPITAL_EXPIRE_FLAG|HAS_CHARTEVENTS_DATA|
+----------+-------+-------------------+--------------------+--------------------+
|        22| 165315|               null|                   0|                   1|
|        23| 152223|               null|                   0|                   1|
|        23| 124321|               null|                   0|                   1|
|        24| 161859|               null|                   0|                   1|
|        25| 129635|               null|                   0|                   1|
|        26| 197661|               null|                   0|                   1|
|        27| 134931|               null|                   0|                   1|
|        28| 162569|               null|                   0|                   1|
|        30| 104557|               null|                   0|                   1|
|   

In [31]:
# Explore unique values of several variables
adm_df.select("ADMISSION_TYPE").distinct().show()

+--------------+
|ADMISSION_TYPE|
+--------------+
|       NEWBORN|
|      ELECTIVE|
|     EMERGENCY|
|        URGENT|
+--------------+



In [32]:
# Explore unique values of several variables
adm_df.select("INSURANCE").distinct().show()

+----------+
| INSURANCE|
+----------+
|Government|
|  Self Pay|
|   Private|
|  Medicaid|
|  Medicare|
+----------+



In [33]:
# Explore unique values of several variables
adm_df.select("ADMISSION_LOCATION").distinct().show()

+--------------------+
|  ADMISSION_LOCATION|
+--------------------+
|PHYS REFERRAL/NOR...|
|** INFO NOT AVAIL...|
|   HMO REFERRAL/SICK|
|TRANSFER FROM HOS...|
|TRANSFER FROM SKI...|
|TRANSFER FROM OTH...|
|EMERGENCY ROOM ADMIT|
|CLINIC REFERRAL/P...|
|TRSF WITHIN THIS ...|
+--------------------+



In [34]:
# Explore unique values of several variables
adm_df.select("ETHNICITY").distinct().show()

+--------------------+
|           ETHNICITY|
+--------------------+
|WHITE - OTHER EUR...|
|UNKNOWN/NOT SPECI...|
|               WHITE|
|      SOUTH AMERICAN|
|    UNABLE TO OBTAIN|
|        ASIAN - THAI|
|HISPANIC/LATINO -...|
|AMERICAN INDIAN/A...|
|      ASIAN - KOREAN|
|  HISPANIC OR LATINO|
|     WHITE - RUSSIAN|
|       BLACK/HAITIAN|
|PATIENT DECLINED ...|
|    ASIAN - FILIPINO|
|   ASIAN - CAMBODIAN|
|HISPANIC/LATINO -...|
|  BLACK/CAPE VERDEAN|
|    ASIAN - JAPANESE|
|      MIDDLE EASTERN|
|HISPANIC/LATINO -...|
+--------------------+
only showing top 20 rows



In [35]:
# Explore unique values of several variables
adm_df.select("ETHNICITY").distinct().count()

41

In [36]:
# Explore unique values of several variables
adm_df.select("MARITAL_STATUS").distinct().show()

+-----------------+
|   MARITAL_STATUS|
+-----------------+
|             null|
|          WIDOWED|
|           SINGLE|
|          MARRIED|
|        SEPARATED|
|UNKNOWN (DEFAULT)|
|         DIVORCED|
|     LIFE PARTNER|
+-----------------+



In [37]:
# Explore unique values of several variables
adm_df.select("LANGUAGE").distinct().show()

+--------+
|LANGUAGE|
+--------+
|    ALBA|
|    *HUN|
|    LAOT|
|    AMER|
|    VIET|
|    CANT|
|    FREN|
|    HIND|
|    *LEB|
|    null|
|    ETHI|
|    *URD|
|    *MAN|
|    *TOY|
|    MAND|
|    *SPA|
|    CAMB|
|    CAPE|
|    *CHI|
|    PTUN|
+--------+
only showing top 20 rows



In [6]:
# Check schema for chart events table
chart_df.printSchema()

root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ICUSTAY_ID: integer (nullable = true)
 |-- ITEMID: integer (nullable = true)
 |-- CHARTTIME: timestamp (nullable = true)
 |-- STORETIME: timestamp (nullable = true)
 |-- CGID: integer (nullable = true)
 |-- VALUE: string (nullable = true)
 |-- VALUENUM: double (nullable = true)
 |-- VALUEUOM: string (nullable = true)
 |-- WARNING: integer (nullable = true)
 |-- ERROR: integer (nullable = true)
 |-- RESULTSTATUS: string (nullable = true)
 |-- STOPPED: string (nullable = true)



In [20]:
# Print first few rows for selected fields
chart_df.select("SUBJECT_ID", "HADM_ID", "ICUSTAY_ID", "ITEMID", 
                "VALUE", "VALUENUM", "VALUEUOM").show(20)

+----------+-------+----------+------+-----+--------+--------+
|SUBJECT_ID|HADM_ID|ICUSTAY_ID|ITEMID|VALUE|VALUENUM|VALUEUOM|
+----------+-------+----------+------+-----+--------+--------+
|        36| 165660|    241249|223834|   15|    15.0|   L/min|
|        36| 165660|    241249|223835|  100|   100.0|    null|
|        36| 165660|    241249|224328|  .37|    0.37|    null|
|        36| 165660|    241249|224329|    6|     6.0|     min|
|        36| 165660|    241249|224330|  2.5|     2.5|    null|
|        36| 165660|    241249|224331|    0|     0.0|   ml/hr|
|        36| 165660|    241249|224332|    3|     3.0|    null|
|        36| 165660|    241249|224663|    8|     8.0|    null|
|        36| 165660|    241249|224665| 1.11|    1.11|    null|
|        36| 165660|    241249|220224|   58|    58.0|    mmHg|
|        36| 165660|    241249|220235|   60|    60.0|    mmHg|
|        36| 165660|    241249|223830| 7.29|    7.29|   units|
|        36| 165660|    241249|224828|    0|     0.0|  

In [21]:
chart_df.select("HADM_ID", "ITEMID", "CHARTTIME", "WARNING", "ERROR",
                "RESULTSTATUS", "STOPPED").show(20)

+-------+------+-------------------+-------+-----+------------+-------+
|HADM_ID|ITEMID|          CHARTTIME|WARNING|ERROR|RESULTSTATUS|STOPPED|
+-------+------+-------------------+-------+-----+------------+-------+
| 165660|223834|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|223835|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|224328|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|224329|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|224330|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|224331|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|224332|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|224663|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|224665|2134-05-12 12:00:00|      0|    0|        null|   null|
| 165660|220224|2134-05-12 12:35:00|      1|    0|        null|   null|
| 165660|220235|2134-05-12 12:35:00|      1|    0|        null| 

In [8]:
# Check schema for chart events table
ditems_df.printSchema()

root
 |-- ROW_ID: integer (nullable = true)
 |-- ITEMID: integer (nullable = true)
 |-- LABEL: string (nullable = true)
 |-- ABBREVIATION: string (nullable = true)
 |-- DBSOURCE: string (nullable = true)
 |-- LINKSTO: string (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- UNITNAME: string (nullable = true)
 |-- PARAM_TYPE: string (nullable = true)
 |-- CONCEPTID: string (nullable = true)



In [23]:
# Print first few rows of selected fields
ditems_df.select("ITEMID", "LABEL", "ABBREVIATION", "DBSOURCE", "LINKSTO").show(20)

+------+--------------------+------------+--------+-----------+
|ITEMID|               LABEL|ABBREVIATION|DBSOURCE|    LINKSTO|
+------+--------------------+------------+--------+-----------+
|   497|Patient controlle...|        null| carevue|chartevents|
|   498|   PCA Lockout (Min)|        null| carevue|chartevents|
|   499|      PCA Medication|        null| carevue|chartevents|
|   500|      PCA Total Dose|        null| carevue|chartevents|
|   501|  PCV Exh Vt (Obser)|        null| carevue|chartevents|
|   927|           Allergy 2|        null| carevue|chartevents|
|   930|                 Ext|        null| carevue|chartevents|
|   935|           Allergy 3|        null| carevue|chartevents|
|   938|      blood cultures|        null| carevue|chartevents|
|   940|          trach care|        null| carevue|chartevents|
|   941|       urine culture|        null| carevue|chartevents|
|   942|      BLOOD CULTURES|        null| carevue|chartevents|
|   944|            Chest PT|        nul

In [9]:
# Check schema for ICU table
icu_df.printSchema()

root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- HADM_ID: integer (nullable = true)
 |-- ICUSTAY_ID: integer (nullable = true)
 |-- DBSOURCE: string (nullable = true)
 |-- FIRST_CAREUNIT: string (nullable = true)
 |-- LAST_CAREUNIT: string (nullable = true)
 |-- FIRST_WARDID: integer (nullable = true)
 |-- LAST_WARDID: integer (nullable = true)
 |-- INTIME: timestamp (nullable = true)
 |-- OUTTIME: timestamp (nullable = true)
 |-- LOS: double (nullable = true)



In [24]:
# Print first few rows of selected columns
icu_df.select("HADM_ID", "ICUSTAY_ID", "DBSOURCE", "FIRST_CAREUNIT", 
              "LAST_CAREUNIT").show(10)

+-------+----------+--------+--------------+-------------+
|HADM_ID|ICUSTAY_ID|DBSOURCE|FIRST_CAREUNIT|LAST_CAREUNIT|
+-------+----------+--------+--------------+-------------+
| 110404|    280836| carevue|          MICU|         MICU|
| 106296|    206613| carevue|          MICU|         MICU|
| 188028|    220345| carevue|           CCU|          CCU|
| 173727|    249196| carevue|          MICU|         SICU|
| 164716|    210407| carevue|           CCU|          CCU|
| 158689|    241507| carevue|          MICU|         MICU|
| 130546|    254851| carevue|          MICU|         MICU|
| 129886|    219649| carevue|           CCU|          CCU|
| 135156|    206327| carevue|           CCU|          CCU|
| 171601|    272866| carevue|          NICU|         NICU|
+-------+----------+--------+--------------+-------------+
only showing top 10 rows



In [26]:
icu_df.select("ICUSTAY_ID", "INTIME", "OUTTIME", "LOS").show(10)

+----------+-------------------+-------------------+------+
|ICUSTAY_ID|             INTIME|            OUTTIME|   LOS|
+----------+-------------------+-------------------+------+
|    280836|2198-02-14 23:27:38|2198-02-18 05:26:11| 3.249|
|    206613|2170-11-05 11:05:29|2170-11-08 17:46:57|3.2788|
|    220345|2128-06-24 15:05:20|2128-06-27 12:32:29|2.8939|
|    249196|2120-08-07 23:12:42|2120-08-10 00:39:04|  2.06|
|    210407|2186-12-25 21:08:04|2186-12-27 12:01:13|1.6202|
|    241507|2141-04-19 06:12:05|2141-04-20 17:52:11|1.4862|
|    254851|2114-06-28 22:28:44|2114-07-07 18:01:16|8.8143|
|    219649|2170-10-07 11:28:53|2170-10-14 14:38:07|7.1314|
|    206327|2147-11-20 09:02:23|2147-11-21 17:08:52|1.3378|
|    272866|2132-10-21 21:11:46|2132-10-22 14:44:48|0.7313|
+----------+-------------------+-------------------+------+
only showing top 10 rows



In [38]:
# Explore unique values of several variables
icu_df.select("FIRST_CAREUNIT").distinct().show()

+--------------+
|FIRST_CAREUNIT|
+--------------+
|          MICU|
|         TSICU|
|          SICU|
|          CSRU|
|           CCU|
|          NICU|
+--------------+



In [39]:
# Explore unique values of several variables
icu_df.select("LAST_CAREUNIT").distinct().show()

+-------------+
|LAST_CAREUNIT|
+-------------+
|         MICU|
|        TSICU|
|         SICU|
|         CSRU|
|          CCU|
|         NICU|
+-------------+



In [10]:
# Check schema for patients table
pat_df.printSchema()

root
 |-- ROW_ID: integer (nullable = true)
 |-- SUBJECT_ID: integer (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- DOB: timestamp (nullable = true)
 |-- DOD: timestamp (nullable = true)
 |-- DOD_HOSP: timestamp (nullable = true)
 |-- DOD_SSN: timestamp (nullable = true)
 |-- EXPIRE_FLAG: integer (nullable = true)



In [27]:
# Print first 5 rows
pat_df.select("SUBJECT_ID", "GENDER", "DOB").show(5)

+----------+------+-------------------+
|SUBJECT_ID|GENDER|                DOB|
+----------+------+-------------------+
|       249|     F|2075-03-13 00:00:00|
|       250|     F|2164-12-27 00:00:00|
|       251|     M|2090-03-15 00:00:00|
|       252|     M|2078-03-06 00:00:00|
|       253|     F|2089-11-26 00:00:00|
+----------+------+-------------------+
only showing top 5 rows



In [28]:
pat_df.select("SUBJECT_ID", "DOD", "DOD_HOSP", "DOD_SSN", "EXPIRE_FLAG").show(5)

+----------+-------------------+-------------------+-------+-----------+
|SUBJECT_ID|                DOD|           DOD_HOSP|DOD_SSN|EXPIRE_FLAG|
+----------+-------------------+-------------------+-------+-----------+
|       249|               null|               null|   null|          0|
|       250|2188-11-22 00:00:00|2188-11-22 00:00:00|   null|          1|
|       251|               null|               null|   null|          0|
|       252|               null|               null|   null|          0|
|       253|               null|               null|   null|          0|
+----------+-------------------+-------------------+-------+-----------+
only showing top 5 rows



In [61]:
# Sanity check on DOD fields
pat_df.filter(pat_df.EXPIRE_FLAG == 0).select("DOD", "DOD_HOSP", "DOD_SSN").distinct().show()

+----+--------+-------+
| DOD|DOD_HOSP|DOD_SSN|
+----+--------+-------+
|null|    null|   null|
+----+--------+-------+



In [40]:
# Explore unique values of several variables
pat_df.select("GENDER").distinct().show()

+------+
|GENDER|
+------+
|     F|
|     M|
+------+



In [ ]:
# Check schema for diagnoses table
diag_df.printSchema()

## Important takeaways from data exploration

### `ADMISSIONS` table

* Each row represents unique admission (`HADM_ID`). Possible for `SUBJECT_ID` to be duplicated (multiple admissions)
* Admission and discharge dates recorded in `ADMITTIME` and `DISCHTIME`
* Patients who died in the hospital indicated from `DEATHTIME`. Another field (with binary flag) is `HOSPITAL_EXPIRE_FLAG`
* Type of admission (`ELECTIVE`, `URGENT`, `NEWBORN`, `EMERGENCY`) available in `ADMISSION_TYPE`
* `ADMISSION_LOCATION` represents previous location of the patient prior to admission (categorical with 9 possible values). Note that text may be truncated
* Demographic variables recorded: `INSURANCE`, `LANGUAGE`, `RELIGION`, `MARITAL_STATUS`, `ETHNICITY`. Sourced from ADT data so may have missingness. `ETHNICITY` may even have discrepancy across admissions
* If patients were admitted to ED, they should have records in `EDREGTIME`, `EDOUTTIME`
* Not recommended to use `DIAGNOSIS` here - discharge diagnoses might be better (`DIAGNOSES_ICD` table)
* Can be linked with `PATIENTS` table using `SUBJECT_ID`

### `CHARTEVENTS` table

* Can be linked to `PATIENTS` (`SUBJECT_ID`), `ADMISSIONS` (`HADM_ID`), `ICUSTAYS` (`ICUSTAY_ID`)
* May need to link `ITEMID` with `D_ITEMS` to identify specific measurements
* If needed, `CHARTTIME` should record the measurement time, although probably using `HADM_ID` should be sufficient
* Values are recorded using `VALUE` (concept-based, identified by `ITEMID`) or `VALUENUM` (numeric value). Unit of measurement is recorded in `VALUEUOM`
* These fields may provide potential flags for abnormal values: `WARNING`, `ERROR` (Metavision), `RESULTSTATUS`, `STOPPED` (CareVue)

### `ICUSTAYS` table
* `ICUSTAY_ID` is unique to a patient ICU stay (assumption: all ICU admissions within 24 hours of each other are grouped). `SUBJECT_ID` and `HADM_ID` are also available
* Database source available in `DBSOURCE` - may be useful when interpreting `CHARTEVENTS` table
* Recorded in and out times for ICU stays (and derived LOS) available in `INTIME`, `OUTTIME`, `LOS`

### `PATIENTS` table

* Unique identifier is given by `SUBJECT_ID`
* Available demographic data: `GENDER`, `DOB`
* IMPORTANT: Patients older than 89 years old are coded as 300 years old based on the deidentified DOB
* Death information: `DOD`, `DOD_HOSP`, `DOD_SSN`. Can be confirmed using binary flag (`EXPIRE_FLAG`)

### `D_ITEMS` table

* As `D_ITEMS` are sourced from two distinct ICU databases (CareVue and Metavision), multiple `ITEMID` for the same concept is possible. Important to search for all possible abbreviations and descriptions of a concept
* `LINKSTO` can be helpful to narrow down if certain concept is relevant to `CHARTEVENTS`
* Key description is available in `LABEL`

### General
* Filter patients using `ADMISSION_TYPE` to include only emergency admissions
* Filter to only their first admission (make a copy as we need the original one to identify re-admission)
* Check availability of chart events data for these patients
* Create a binary flag for ED
* Ensure that `DOD` in `PATIENTS` table is consistent with `DOD_HOSP` or `DOD_SSN`

## Filtering relevant data

In [41]:
emergency_adm_df = adm_df.filter(adm_df.ADMISSION_TYPE == "EMERGENCY")
emergency_adm_df = emergency_adm_df.select(
    "SUBJECT_ID", "HADM_ID", "ADMITTIME", "DISCHTIME", "DEATHTIME", "HOSPITAL_EXPIRE_FLAG", 
    "ADMISSION_LOCATION", "INSURANCE", "LANGUAGE", "MARITAL_STATUS", "ETHNICITY", 
    "EDREGTIME", "EDOUTTIME"
)
emergency_adm_df.count()

42071

In [46]:
# Number of unique patients
emergency_adm_df.select("SUBJECT_ID").distinct().count()

32610

In [52]:
# Sanity check: Ensure no recorded death date for patients who didn't die
print(emergency_adm_df.filter(emergency_adm_df.HOSPITAL_EXPIRE_FLAG == 0).count())
print(emergency_adm_df.filter(emergency_adm_df.DEATHTIME.isNull()).count())

36637
36637


In [55]:
# Assess missingness for some variables
print(emergency_adm_df.filter(emergency_adm_df.MARITAL_STATUS.isNull()).count())

2342


In [57]:
print(emergency_adm_df.filter(emergency_adm_df.ETHNICITY.contains("UNKNOWN")).count())

3106


In [58]:
print(emergency_adm_df.filter(emergency_adm_df.ETHNICITY.contains("UNABLE")).count())

690


In [59]:
print(emergency_adm_df.filter(emergency_adm_df.ADMISSION_LOCATION.contains(
    "INFO NOT AVAIL")).count())

5


In [66]:
# Check number of unique items in chart events
chart_df.join(emergency_adm_df.select("HADM_ID"), 
              emergency_adm_df.HADM_ID == chart_df.HADM_ID, "inner").count()

238855068

In [65]:
chart_df.count()

330712483

In [67]:
chart_df.join(emergency_adm_df.select("HADM_ID"), 
              emergency_adm_df.HADM_ID == chart_df.HADM_ID, "inner").select(
    "ITEMID").distinct().count()

5214

Next steps:
* Filter first admissions only and extract 90-day readmission data **DONE**
* Exclude patients who died in the hospital **DONE**
* Create binary flag for ED visits during index stay **DONE**
* Create binary flag for ICU visits during index stay **EXPLORED**
* **NEXT STEP** Extract discharge diagnoses from `DIAGNOSES_ICD` using `ICD9_CODE` and `SEQ_NUM` (to identify primary/secondary diagnoses) - linkable using `HADM_ID`
* **Optional:** Look at common diagnoses and identify commonly measured chart items to be extracted
* Standardise recording of missing values in each column **PARTIALLY DONE**
* **NEXT STEP** Clean up process to save and load CSV files generated by PySpark

In [129]:
# Identify first admissions for all patients
windowSubj = Window.partitionBy("SUBJECT_ID").orderBy(col("ADMITTIME"))
first_adm_df = emergency_adm_df.withColumn("row", row_number().over(windowSubj)).filter(
    col("row") == 1).drop("row")

In [130]:
# Sanity check on a single patient
emergency_adm_df.filter(col("SUBJECT_ID") == 249).select(
    "HADM_ID", "ADMITTIME", "DISCHTIME").show()

+-------+-------------------+-------------------+
|HADM_ID|          ADMITTIME|          DISCHTIME|
+-------+-------------------+-------------------+
| 116935|2149-12-17 20:41:00|2149-12-31 14:55:00|
| 149546|2155-02-03 20:16:00|2155-02-14 11:15:00|
| 158975|2156-04-27 15:33:00|2156-05-14 15:30:00|
+-------+-------------------+-------------------+



In [131]:
first_adm_df.filter(col("SUBJECT_ID") == 249).select(
    "HADM_ID", "ADMITTIME", "DISCHTIME").show()

+-------+-------------------+-------------------+
|HADM_ID|          ADMITTIME|          DISCHTIME|
+-------+-------------------+-------------------+
| 116935|2149-12-17 20:41:00|2149-12-31 14:55:00|
+-------+-------------------+-------------------+



In [132]:
# Change column names for index admission info
first_adm_df = first_adm_df.withColumnRenamed("ADMITTIME", "INDEX_ADMITTIME")
first_adm_df = first_adm_df.withColumnRenamed("DISCHTIME", "INDEX_DISCHTIME")
first_adm_df = first_adm_df.withColumnRenamed("HADM_ID", "INDEX_HADM_ID")

In [133]:
# Join with the bigger table to identify readmissions
first_adm_df_joined = first_adm_df.select(
    "SUBJECT_ID", "INDEX_HADM_ID", "INDEX_DISCHTIME").join(
    emergency_adm_df.select("SUBJECT_ID", "HADM_ID", "ADMITTIME"), 
    first_adm_df.SUBJECT_ID == emergency_adm_df.SUBJECT_ID
)

# Remove index admission records from consideration
first_adm_df_joined = first_adm_df_joined.filter(col("INDEX_HADM_ID") != col("HADM_ID"))

first_adm_df_joined.count()

9461

In [134]:
first_adm_df_joined.show(20)

+----------+-------------+-------------------+----------+-------+-------------------+
|SUBJECT_ID|INDEX_HADM_ID|    INDEX_DISCHTIME|SUBJECT_ID|HADM_ID|          ADMITTIME|
+----------+-------------+-------------------+----------+-------+-------------------+
|        21|       109451|2134-09-24 16:15:00|        21| 111970|2135-01-30 20:50:00|
|        34|       115799|2186-07-20 16:00:00|        34| 144319|2191-02-23 05:23:00|
|        36|       182104|2131-05-08 14:00:00|        36| 122659|2131-05-12 19:49:00|
|        68|       170467|2174-01-03 18:30:00|        68| 108329|2174-01-04 22:21:00|
|        85|       116630|2162-03-10 13:15:00|        85| 112077|2167-07-25 18:49:00|
|        94|       183686|2176-02-29 17:45:00|        94| 140037|2176-09-02 14:22:00|
|       103|       130744|2144-08-20 11:15:00|       103| 133550|2144-08-30 23:09:00|
|       105|       161160|2189-02-02 16:40:00|       105| 128744|2189-02-21 01:45:00|
|       107|       191941|2115-02-21 16:30:00|       1

In [135]:
# Calculate time difference and identify 90-day readmission
first_adm_df_joined = first_adm_df_joined.withColumn(
    "DAYS_FROM_INDEX", 
    (unix_timestamp("ADMITTIME") - unix_timestamp("INDEX_DISCHTIME"))/(3600*24)
)

first_adm_df_joined = first_adm_df_joined.withColumn(
    "90DAYREADM", col("DAYS_FROM_INDEX") <= 90
)

In [136]:
first_adm_df_joined.filter(first_adm_df_joined["90DAYREADM"] == True).count()

2646

In [137]:
first_adm_df_joined.filter(first_adm_df_joined["90DAYREADM"] == True).select(
    "INDEX_HADM_ID", "INDEX_DISCHTIME", "ADMITTIME").show(20)

+-------------+-------------------+-------------------+
|INDEX_HADM_ID|    INDEX_DISCHTIME|          ADMITTIME|
+-------------+-------------------+-------------------+
|       182104|2131-05-08 14:00:00|2131-05-12 19:49:00|
|       170467|2174-01-03 18:30:00|2174-01-04 22:21:00|
|       130744|2144-08-20 11:15:00|2144-08-30 23:09:00|
|       161160|2189-02-02 16:40:00|2189-02-21 01:45:00|
|       198214|2119-11-05 17:00:00|2119-11-14 12:00:00|
|       110545|2106-05-26 16:15:00|2106-06-17 19:51:00|
|       159223|2183-11-11 13:25:00|2184-01-21 21:20:00|
|       137477|2168-03-16 15:45:00|2168-04-24 20:29:00|
|       160697|2157-01-19 14:58:00|2157-03-07 11:08:00|
|       109185|2166-09-12 14:41:00|2166-10-02 15:36:00|
|       106909|2176-02-24 16:23:00|2176-04-10 20:45:00|
|       119446|2140-08-19 17:00:00|2140-09-29 12:55:00|
|       146586|2149-11-06 15:24:00|2149-12-19 02:34:00|
|       108923|2151-04-13 16:10:00|2151-06-23 22:18:00|
|       135591|2132-08-13 17:41:00|2132-08-31 17

In [138]:
# Join the readmission flag back to the first admission table

first_adm_df_joined = first_adm_df_joined.withColumnRenamed("INDEX_HADM_ID", "INDEX_HADM_ID2")
first_adm_df = first_adm_df.join(
    first_adm_df_joined.select("INDEX_HADM_ID2", "90DAYREADM").filter(
        first_adm_df_joined["90DAYREADM"] == True).dropDuplicates(), 
    first_adm_df.INDEX_HADM_ID == first_adm_df_joined.INDEX_HADM_ID2,
    "left"
)

In [139]:
first_adm_df.count()

32610

In [140]:
first_adm_df.filter(first_adm_df["90DAYREADM"] == True).count()

2290

In [141]:
# Drop redundant columns and fill missing readmission flags
first_adm_df = first_adm_df.drop("INDEX_HADM_ID2")
first_adm_df = first_adm_df.fillna({"90DAYREADM": False})

In [142]:
first_adm_df.filter(first_adm_df["90DAYREADM"] == True).count()

2290

In [143]:
# Check patients who died in hospital and readmitted at the same time
to_check = first_adm_df.filter(
    (first_adm_df["HOSPITAL_EXPIRE_FLAG"] == 1) & (first_adm_df["90DAYREADM"] == True))

In [144]:
to_check.count()

31

In [150]:
to_check.select("INDEX_HADM_ID", "INDEX_ADMITTIME", "INDEX_DISCHTIME", "DEATHTIME").show()

+-------------+-------------------+-------------------+-------------------+
|INDEX_HADM_ID|    INDEX_ADMITTIME|    INDEX_DISCHTIME|          DEATHTIME|
+-------------+-------------------+-------------------+-------------------+
|       171956|2138-05-04 21:33:00|2138-05-12 12:00:00|2138-05-12 12:00:00|
|       173380|2188-12-08 17:37:00|2188-12-16 13:12:00|2188-12-16 13:12:00|
|       166578|2157-02-24 01:17:00|2157-02-27 05:18:00|2157-02-27 05:18:00|
|       134011|2158-02-16 22:03:00|2158-02-17 08:00:00|2158-02-17 08:00:00|
|       133293|2164-10-14 01:57:00|2164-10-16 16:30:00|2164-10-16 16:30:00|
|       189200|2160-08-26 17:58:00|2160-08-27 08:50:00|2160-08-27 08:50:00|
|       148592|2163-01-20 18:39:00|2163-01-24 08:00:00|2163-01-26 08:00:00|
|       178629|2153-05-27 04:35:00|2153-05-27 08:00:00|2153-05-27 08:00:00|
|       141260|2140-01-12 04:45:00|2140-01-13 10:54:00|2140-01-13 10:54:00|
|       165450|2131-02-10 00:09:00|2131-02-11 14:45:00|2131-02-12 14:45:00|
|       1138

In [160]:
to_check = to_check.join(first_adm_df_joined.select("INDEX_HADM_ID2", "DAYS_FROM_INDEX"), 
                         to_check.INDEX_HADM_ID == first_adm_df_joined.INDEX_HADM_ID2,
                         "left")

In [161]:
to_check.count()

31

In [166]:
to_check.describe("DAYS_FROM_INDEX").show()

+-------+-------------------+
|summary|    DAYS_FROM_INDEX|
+-------+-------------------+
|  count|                 31|
|   mean|0.13637992831541218|
| stddev|0.27503826645348745|
|    min|-0.3055555555555556|
|    max|  1.207638888888889|
+-------+-------------------+



In [145]:
# Remove patients who died in the hospital
first_adm_df = first_adm_df.filter(first_adm_df["HOSPITAL_EXPIRE_FLAG"] != 1)

print("Number of patients left: {}".format(first_adm_df.count()))

Number of patients left: 28404


In [146]:
first_adm_df.filter(first_adm_df["90DAYREADM"] == True).count()

2259

In [167]:
# Create binary flag for ED visits
first_adm_df = first_adm_df.withColumn("ED_FLAG", col("EDREGTIME").isNotNull())

In [169]:
first_adm_df.filter(col("ED_FLAG") == True).count()

19809

In [175]:
first_adm_df.printSchema()

root
 |-- SUBJECT_ID: integer (nullable = true)
 |-- INDEX_HADM_ID: integer (nullable = true)
 |-- INDEX_ADMITTIME: timestamp (nullable = true)
 |-- INDEX_DISCHTIME: timestamp (nullable = true)
 |-- DEATHTIME: timestamp (nullable = true)
 |-- HOSPITAL_EXPIRE_FLAG: integer (nullable = true)
 |-- ADMISSION_LOCATION: string (nullable = true)
 |-- INSURANCE: string (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- EDREGTIME: timestamp (nullable = true)
 |-- EDOUTTIME: timestamp (nullable = true)
 |-- 90DAYREADM: boolean (nullable = false)
 |-- ED_FLAG: boolean (nullable = false)



In [171]:
# Link with ICU data to identify patients with ICU visits
# NOTE: Decided not to include as most patients seem to have one
icu_adm_linked = first_adm_df.join(icu_df.select("HADM_ID", "ICUSTAY_ID"), 
                                   icu_df.HADM_ID == first_adm_df.INDEX_HADM_ID, 
                                   "inner")
icu_index_adm = icu_adm_linked.groupBy("HADM_ID").count().withColumnRenamed(
    "count", "ICU_COUNT")

icu_index_adm.count()

28159

In [196]:
# Remove date related columns and IDs (not needed for modelling)
first_adm_df1 = first_adm_df.drop(
    "INDEX_ADMITTIME", "INDEX_DISCHTIME", "INDEX_HADM_ID", "SUBJECT_ID", 
    "DEATHTIME", "HOSPITAL_EXPIRE_FLAG", "EDREGTIME", "EDOUTTIME"
)

In [197]:
# Standardise recording of missing values
first_adm_df1 = first_adm_df1.withColumn(
    "ADMISSION_LOCATION_CLEAN", 
    when(~col("ADMISSION_LOCATION").contains("INFO NOT AVAIL"), col("ADMISSION_LOCATION"))
)
first_adm_df1 = first_adm_df1.withColumn(
    "MARITAL_STATUS_CLEAN", 
    when(~col("MARITAL_STATUS").contains("UNKNOWN"), col("MARITAL_STATUS"))
)

In [198]:
first_adm_df1.select("ADMISSION_LOCATION_CLEAN").distinct().show()

+------------------------+
|ADMISSION_LOCATION_CLEAN|
+------------------------+
|    PHYS REFERRAL/NOR...|
|                    null|
|       HMO REFERRAL/SICK|
|    TRANSFER FROM HOS...|
|    TRANSFER FROM SKI...|
|    TRANSFER FROM OTH...|
|    EMERGENCY ROOM ADMIT|
|    TRSF WITHIN THIS ...|
|    CLINIC REFERRAL/P...|
+------------------------+



In [199]:
first_adm_df1.select("MARITAL_STATUS_CLEAN").distinct().show()

+--------------------+
|MARITAL_STATUS_CLEAN|
+--------------------+
|                null|
|             WIDOWED|
|              SINGLE|
|        LIFE PARTNER|
|             MARRIED|
|           SEPARATED|
|            DIVORCED|
+--------------------+



In [200]:
# Clean up column names
first_adm_df1 = first_adm_df1.drop("MARITAL_STATUS", "ADMISSION_LOCATION")
first_adm_df1 = first_adm_df1.withColumnRenamed("MARITAL_STATUS_CLEAN", "MARITAL_STATUS")
first_adm_df1 = first_adm_df1.withColumnRenamed(
    "ADMISSION_LOCATION_CLEAN", "ADMISSION_LOCATION"
)

In [201]:
first_adm_df1.printSchema()

root
 |-- INSURANCE: string (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- 90DAYREADM: boolean (nullable = false)
 |-- ED_FLAG: boolean (nullable = false)
 |-- ADMISSION_LOCATION: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)



In [203]:
# Save dataframe as CSV
first_adm_df1.write.option("header", True).csv(mimic_dir + "processed_table")

In [204]:
test = pd.read_csv(
    mimic_dir + "processed_table/part-00000-0ba001e1-3d19-4411-b020-3389613dd064-c000.csv"
)

In [205]:
test.shape

(28404, 7)

In [206]:
test.head()

,INSURANCE,LANGUAGE,ETHNICITY,90DAYREADM,ED_FLAG,ADMISSION_LOCATION,MARITAL_STATUS
0,Medicare,NaN,WHITE,False,True,EMERGENCY ROOM ADMIT,WIDOWED
1,Medicare,NaN,BLACK/AFRICAN AMERICAN,False,True,EMERGENCY ROOM ADMIT,DIVORCED
2,Private,NaN,WHITE,False,True,EMERGENCY ROOM ADMIT,SINGLE
3,Medicare,NaN,WHITE,True,True,EMERGENCY ROOM ADMIT,MARRIED
4,Private,NaN,WHITE,False,False,TRANSFER FROM HOSP/EXTRAM,MARRIED
